In [2]:
import os, pandas as pd, numpy as np, matplotlib.pyplot as plt
import networkx, scprep, magic, torch, pykeen, phate
import rpy2, rpy2.situation
from rpy2.robjects import r, pandas2ri

In [3]:
print(os.getcwd())
data_dir = f"{os.getcwd()}/data"
kge_dir = f"{os.getcwd()}/KGEs"

/gpfs/ysm/home/fc489/yaleCollab


Ligand **-***sender|receiver***->** Receptor **-->** Pathway **-->** Target 

* L-R interactions from NichetNet (or anything else really)
* Target gene list from Nichenet too. 
* Pathways from Reactome. Define receptor/target-pathway edge based on ReactomeDB 

We will have to be carefull with the targets added, as in the previous nichenetKG they were the vast majority of nodes.


# Data acquisition

In [4]:
for i in rpy2.situation.iter_info(): # Print Rpy2 info
    print(i)

pandas2ri.activate()
%load_ext rpy2.ipython

rpy2 version:
3.5.1
Python version:
3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:58:50) 
[GCC 10.3.0]
Looking for R's HOME:
    Environment variable R_HOME: /gpfs/ysm/project/krishnaswamy_smita/fc489/conda_envs/collab/lib/R
    Calling `R RHOME`: /gpfs/ysm/project/krishnaswamy_smita/fc489/conda_envs/collab/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:

R version:
    In the PATH: R version 4.1.3 (2022-03-10) -- "One Push-Up"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/gpfs/ysm/project/krishnaswamy_smita/fc489/conda_envs/collab/lib/R/include']
  libraries:
  ['R', 'pcre2-8', 'lzma', 'bz2', 'z', 'rt', 'dl', 'm', 'iconv', 'icuuc', 'icui18n']
  library_dirs:
  ['/gpfs/ysm/project/krishnaswamy_smita/fc489/conda_envs/collab/lib', '/gpfs/ysm/project/krishnaswamy_smita/fc489/conda_envs/collab/lib/R/lib', '/gpfs/ysm/project/krishnaswamy_smita/fc489/conda_envs/c

## REACTOME

In [31]:
#LIst of pathways of interest (2nd level pathways)
dREACTpaths = pd.read_csv(f"{data_dir}/DBs/ReactomePathwaysRelation.txt", sep="\t", header=None)
dREACTpaths = dREACTpaths.loc[dREACTpaths[0].isin([
                "R-HSA-162582",#Signal transduction
                "R-HSA-5357801",#Programmed cell death
                "R-HSA-73894",#DNA repair
                "R-HSA-8953897",#Cell response to stress
                "R-HSA-1500931",#Cell-cell commns
                "R-HSA-1640170"#Cell cycle
                                                    ]),1]
dREACTpaths.shape

(35,)

In [89]:
dREACTpe = pd.read_csv(f"{data_dir}/DBs/Ensembl2Reactome_PE_All_Levels.txt", sep="\t", header=None)
dREACTpe = dREACTpe.loc[dREACTpe[0].str.startswith("ENSG")|dREACTpe[0].str.startswith("ENSP"),[1,2,3,5]].drop_duplicates()
dREACTpe = dREACTpe.loc[dREACTpe[3].isin(dREACTpaths)]
dREACTpe = dREACTpe.loc[dREACTpe[2].str.match("^\S*[A-Z0-9]+\S*[A-Z0-9]+(\sgene)?(\(\S+\))? \[")][[2,3,5]].drop_duplicates()
print(dREACTpe[3].nunique())
dREACTpe

35


,2,3,5
1533488,GCLC gene [nucleoplasm],R-HSA-2262752,Cellular responses to stress
1533489,GCLC [cytosol],R-HSA-2262752,Cellular responses to stress
1533535,NFYA [nucleoplasm],R-HSA-2262752,Cellular responses to stress
1533688,CFTR [plasma membrane],R-HSA-9716542,"Signaling by Rho GTPases, Miro GTPases and RHO..."
1533689,CFTR [Golgi-associated vesicle membrane],R-HSA-9716542,"Signaling by Rho GTPases, Miro GTPases and RHO..."
...,...,...,...
2068808,miR-24-2 [cytosol],R-HSA-2262752,Cellular responses to stress
2068823,miR-24-1 [cytosol],R-HSA-2262752,Cellular responses to stress
2070835,ARHGAP11B [cytosol],R-HSA-9716542,"Signaling by Rho GTPases, Miro GTPases and RHO..."
2075559,NPBWR1 [plasma membrane],R-HSA-372790,Signaling by GPCR


## Ligand-Receptors

NicheNet

In [5]:
%%R -o dNNlr
#Directly load RDS object from NicheNet's zenodo repo
dNNlr = readRDS(url("https://zenodo.org/record/3260758/files/lr_network.rds"))
# dNNgr = readRDS(url("https://zenodo.org/record/3260758/files/gr_network.rds"))
# dNNgrt = readRDS(url("https://zenodo.org/record/3260758/files/tf_target_matrix.rds"))
# dNNlt = readRDS(url("https://zenodo.org/record/3260758/files/ligand_target_matrix.rds"))

In [6]:
print("Ligand-Receptor\n",dNNlr.describe())
# print("TF-Target\n",dNNgr.describe())
# print("TF-Target 2\n",dNNgrt.shape,dNNgrt)
# print("Ligand-Target\n",dNNlt.shape,dNNlt)
dNNlr

Ligand-Receptor
          from     to  source        database
count   12651  12651   12651           12651
unique    688    857      14               5
top       AGT   FPR2  ppi_lr  ppi_prediction
freq      192     79    5956            6304


,from,to,source,database
1,CXCL1,CXCR2,kegg_cytokines,kegg
2,CXCL2,CXCR2,kegg_cytokines,kegg
3,CXCL3,CXCR2,kegg_cytokines,kegg
4,CXCL5,CXCR2,kegg_cytokines,kegg
5,PPBP,CXCR2,kegg_cytokines,kegg
...,...,...,...,...
12647,CXCL6,PIGR,ppi_lr_go,ppi_prediction_go
12648,RLN1,GPR25,ppi_lr_go,ppi_prediction_go
12649,RLN1,RAMP3,ppi_lr_go,ppi_prediction_go
12650,PIK3CB,TNFRSF10A,ppi_bidir_r_go,ppi_prediction_go


On the L-R side, NicheNet seems to have a higher number of interactions
modelled in. We can just use them *as is*.

The tf_target and ligand_target matrix objects don't look to be as useful since the targets they have are actually, if anything, somewaht longer than the targets in the gene regulation database (uncomment .md below to see)
Manual exploration of the tf_target database revealed that all genes with bio meaning I could think of are present in this list.
Afterall,we have 25k genes in here. This is thus, not useful at all.

<!-- Ligand-Receptor
          from     to  source        database
count   12651  12651   12651           12651
unique    688    857      14               5
top       AGT   FPR2  ppi_lr  ppi_prediction
freq      192     79    5956            6304

TF-Target
            from       to              source        database
count   3592299  3592299             3592299         3592299
unique     4486    25103                  20               8
top         YY1   CDKN1A  harmonizome_ENCODE  harmonizome_gr
freq      47472      921             1564116         2953691

TF-Target 2
 (25345, 4486) [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
 
Ligand-Target
 (25345, 688) [[0.00035343 0.00040413 0.00037299 ... 0.00028765 0.00036615 0.00032276]
 [0.00016509 0.00015092 0.00015836 ... 0.00014044 0.0001839  0.00017542]
 [0.00057872 0.00045963 0.00038959 ... 0.00053559 0.00051685 0.00040327]
 ...
 [0.00043514 0.00052277 0.00040791 ... 0.00043696 0.00092245 0.00045744]
 [0.00041811 0.00049604 0.00045687 ... 0.00036184 0.00059071 0.00040423]
 [0.00050178 0.00055816 0.00048933 ... 0.00053238 0.00099519 0.00057892]] -->
 

CellChat

In [7]:
dCClr = pd.read_csv(f"{data_dir}/dCellChat_hsapiens.csv.tar.gz",header=None)
dCClr

,0,1,2,3
0,TGFB1,LRinteraction,TGFBR1,TGFb
1,TGFB1,LRinteraction,TGFBR2,TGFb
2,TGFB2,LRinteraction,TGFBR1,TGFb
3,TGFB2,LRinteraction,TGFBR2,TGFb
4,TGFB3,LRinteraction,TGFBR1,TGFb
...,...,...,...,...
1976,VCAM1,LRinteraction,ITGA4,VCAM
1977,VCAM1,LRinteraction,ITGB1,VCAM
1978,VCAM1,LRinteraction,ITGA9,VCAM
1979,VCAM1,LRinteraction,ITGB7,VCAM


Merge NicheNet and CellChat LR databases

In [8]:
dNNlr = pd.DataFrame({
            "head":dNNlr["from"],
            "relation":np.repeat("LR", dNNlr.shape[0]),
            "tail":dNNlr["to"]
        })
dCClr = pd.DataFrame({
            "head":dCClr[0],
            "relation":np.repeat("LR", dCClr.shape[0]),
            "tail":dCClr[2]
        })
dLR = pd.concat([dNNlr, dCClr]).drop_duplicates().reset_index(drop=True)
del dNNlr,dCClr
dLR

,head,relation,tail
0,CXCL1,LR,CXCR2
1,CXCL2,LR,CXCR2
2,CXCL3,LR,CXCR2
3,CXCL5,LR,CXCR2
4,PPBP,LR,CXCR2
...,...,...,...
12927,THY1,LR,ITGB2
12928,THY1,LR,ITGAX
12929,THY1,LR,ITGAV
12930,THY1,LR,ITGB3


## PTMs
Get the PTMs from the cytof dataset. Manually curate them and ensure they match to proper REACTOME PE IDS.

In [21]:
dCyt = pd.read_csv(f"{data_dir}/Omics/dWTfib_cytof.txt",sep="\t").filter(regex = "^\d+|^file_identifier")
dCyt

,89Y_pHH3_S28,111Cd_CD44_v2,113In_CEACAM1_CD66a_v2,114Cd_CK18_v2,115In_Pan-CK_v5,122Te_Barcode,123Te_Barcode,124Te_Barcode,125Te_Barcode,126Te_Barcode,...,172Yb_pS6_S235_236_v2,173Yb_PDPN_v2,174Yb_RFP,175Lu_Vimentin,176Yb_Cyclin_B1_2_v2,191Ir_DNA_1,193Ir_DNA_2,194Pt_Cisplatin,209Bi_CK18,file_identifier
0,0.000000,1990.980957,18.211060,2928.800781,1973.129028,771.304932,14.819576,563.619202,699.982117,25.099878,...,11688.049805,504.713531,29.877979,4299.611816,200.047882,63777.085938,121983.914062,18089.599609,1510.892578,Pro_WTfib_fib
1,0.000000,825.002075,1.621021,8.841931,45.782940,400.186676,53.228016,331.107513,406.801514,51.104706,...,5725.374512,2165.157471,213.488480,1535.135986,132.647461,18011.433594,32582.429688,1185.106079,0.000000,Pro_WTfib_fib
2,0.545875,23.752832,2.710696,75.029808,224.630737,6442.486816,119.965919,6082.173828,6060.972168,700.094177,...,0.000000,515.552185,825.009094,0.000000,37.127651,36602.984375,72815.867188,14817.324219,43.982540,Pro_WTfib_fib
3,1.687439,993.330872,4.313036,18.532772,19.012230,179.756882,25.230639,174.673813,156.020660,21.305735,...,31.062332,453.606934,234.555603,32.569691,10.049050,25414.136719,47861.183594,1220.816040,1.686805,Pro_WTfib_fib
4,0.000000,55.538124,1.648003,29.172148,8.418515,384.167419,13.115532,243.553711,243.389740,24.149267,...,21.605457,161.520538,49.635128,0.000000,0.000000,13607.113281,23980.585938,1471.665405,0.000000,Pro_WTfib_fib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23944,0.575287,0.000000,29.917067,1177.036377,1878.979126,18780.707031,347.416901,17428.015625,18221.324219,1969.474243,...,5.057156,105.512360,56.930191,0.000000,128.641403,16511.298828,31044.115234,7563.200195,516.480652,Pro_WTfib_epi
23945,8.733110,51.412457,316.347076,2758.709229,1413.775513,17464.484375,319.504669,16184.413086,17701.402344,1702.833252,...,2.509986,6.395815,48.340462,3.997385,336.654144,15596.995117,32046.751953,1247.258301,2554.626221,Pro_WTfib_epi
23946,0.000000,32.676712,47.151875,218.732193,142.701355,4548.200684,99.833855,4417.928711,4601.003906,413.944672,...,0.000000,0.000000,67.100281,0.000000,836.829224,12291.399414,25326.033203,1548.010376,302.388184,Pro_WTfib_epi
23947,2.054595,5.049978,3.555531,236.509842,129.904480,41.158947,3.220307,73.682106,52.007206,9.751757,...,0.000000,0.000000,15.115691,0.000000,27.275177,2755.387939,6044.491699,109.323814,568.037659,Pro_WTfib_epi


In [24]:
dCyt.columns

Index(['89Y_pHH3_S28', '111Cd_CD44_v2', '113In_CEACAM1_CD66a_v2',
       '114Cd_CK18_v2', '115In_Pan-CK_v5', '122Te_Barcode', '123Te_Barcode',
       '124Te_Barcode', '125Te_Barcode', '126Te_Barcode', '127I_IdU',
       '128Te_Barcode', '130Te_Barcode', '140Ce_EQ_Beads',
       '141Pr_pPDK1_S241_v4', '142Nd_cCaspase_3_D175_v3', '143Nd_NaK_ATPase',
       '144Nd_pMEK1_2_S221_v3', '145Nd_CD34', '146Nd_pMKK4_SEK1_S257_v3',
       '147Sm_pBTK_Y511_v4', '148Nd_pSRC_Y418_v5', '149Sm_p4EBP1_T37_T46_v2',
       '150Nd_pRB_S807_S811_v3', '152Sm_pAKT_T308_v7', '153Eu_pCREB_S133_v8',
       '154Sm_pSMAD1_5_9_v5', '155Gd_pAKT_S473_v6', '156Gd_pNF-kB_p65_S529_v4',
       '157Gd_pMKK3_MKK6_S189_207_v3', '158Gd_pP38_MAPK_T180_Y182_v3',
       '159Tb_pMAPKAPK2_T334_v2', '160Gd_pAMPKa_T172_v3', '161Dy_pBAD_S112_v4',
       '162Dy_LRIG1_1_v2', '163Dy_pP90RSK_T359_Hu_v5', '164Dy_Alpha-SMA_2',
       '165Ho_Beta-catenin_Active_1_v4', '166Er_pGSK-3b_S9_v2',
       '167Er_pERK1_2_T202_Y204_v2', '168Er_pSMAD

In [30]:
dPTMs = [
    '89Y_pHH3_S28',
    '141Pr_pPDK1_S241_v4',
    '142Nd_cCaspase_3_D175_v3',
    '144Nd_pMEK1_2_S221_v3',
    '146Nd_pMKK4_SEK1_S257_v3',
    '147Sm_pBTK_Y511_v4', '148Nd_pSRC_Y418_v5', '149Sm_p4EBP1_T37_T46_v2',
    '150Nd_pRB_S807_S811_v3', '152Sm_pAKT_T308_v7', '153Eu_pCREB_S133_v8',
    '154Sm_pSMAD1_5_9_v5', '155Gd_pAKT_S473_v6', '156Gd_pNF-kB_p65_S529_v4',
       '157Gd_pMKK3_MKK6_S189_207_v3', '158Gd_pP38_MAPK_T180_Y182_v3',
       '159Tb_pMAPKAPK2_T334_v2', '160Gd_pAMPKa_T172_v3', '161Dy_pBAD_S112_v4',
    '163Dy_pP90RSK_T359_Hu_v5',
    '166Er_pGSK-3b_S9_v2',
    '167Er_pERK1_2_T202_Y204_v2', '168Er_pSMAD2_3_v4', '169Tm_pSTAT3_Y705',
    '170Er_pMEK1_2_S221_v2',
    '172Yb_pS6_S235_236_v2'
    
]
dCyt = dCyt.filter(dPTMs)
dCyt.columns

Index(['89Y_pHH3_S28', '141Pr_pPDK1_S241_v4', '142Nd_cCaspase_3_D175_v3',
       '144Nd_pMEK1_2_S221_v3', '146Nd_pMKK4_SEK1_S257_v3',
       '147Sm_pBTK_Y511_v4', '148Nd_pSRC_Y418_v5', '149Sm_p4EBP1_T37_T46_v2',
       '150Nd_pRB_S807_S811_v3', '152Sm_pAKT_T308_v7', '153Eu_pCREB_S133_v8',
       '154Sm_pSMAD1_5_9_v5', '155Gd_pAKT_S473_v6', '156Gd_pNF-kB_p65_S529_v4',
       '157Gd_pMKK3_MKK6_S189_207_v3', '158Gd_pP38_MAPK_T180_Y182_v3',
       '159Tb_pMAPKAPK2_T334_v2', '160Gd_pAMPKa_T172_v3', '161Dy_pBAD_S112_v4',
       '163Dy_pP90RSK_T359_Hu_v5', '166Er_pGSK-3b_S9_v2',
       '167Er_pERK1_2_T202_Y204_v2', '168Er_pSMAD2_3_v4', '169Tm_pSTAT3_Y705',
       '170Er_pMEK1_2_S221_v2', '172Yb_pS6_S235_236_v2'],
      dtype='object')

In [55]:
dREACTpe[2]

0                          acpA [cytosol]
1                          acpA [cytosol]
2                          acpA [cytosol]
3                          acpA [cytosol]
4                          acpA [cytosol]
                        ...              
5751755    phospho-p-T19-MYL12B [cytosol]
5751756                  MYL12B [cytosol]
5751757                    MYL9 [cytosol]
5751758      phospho-p-T19-MYL9 [cytosol]
5751759    phospho-p-T19-MYL12B [cytosol]
Name: 2, Length: 5751760, dtype: object

In [90]:
dREACTpe.loc[dREACTpe[2].str.contains("H3F3A")]

,2,3,5
1710234,Me2K5-H3F3A [nucleoplasm],R-HSA-1500620,Meiosis
1710235,Me3K5-H3F3A [nucleoplasm],R-HSA-1500620,Meiosis
1710236,H3F3A [nucleoplasm],R-HSA-157118,Signaling by NOTCH
1710237,AcK10-H3F3A [nucleoplasm],R-HSA-157118,Signaling by NOTCH
1710270,H3F3A [nucleoplasm],R-HSA-195721,Signaling by WNT
1710271,Me3K5-H3F3A [nucleoplasm],R-HSA-195721,Signaling by WNT
1710291,H3F3A [nucleoplasm],R-HSA-2262752,Cellular responses to stress
1710292,Me3K-28-H3F3A [nucleoplasm],R-HSA-2262752,Cellular responses to stress
1710293,Me2K-10-H3F3A [nucleoplasm],R-HSA-2262752,Cellular responses to stress
1710346,Me2K5-H3F3A [nucleoplasm],R-HSA-69278,"Cell Cycle, Mitotic"


In [91]:
dREACTpe.loc[dREACTpe[2].str.contains("PDK1")]

,2,3,5
1816163,PDK1 [mitochondrial matrix],R-HSA-9006931,Signaling by Nuclear Receptors


In [92]:
dREACTpe.loc[dREACTpe[2].str.contains("CASP3")]

,2,3,5
1850039,CASP3(176-277) [cytosol],R-HSA-109581,Apoptosis
1850041,CASP3(29-175) [cytosol],R-HSA-109581,Apoptosis
1850043,CASP3(176-277) [nucleoplasm],R-HSA-109581,Apoptosis
1850044,CASP3(29-175) [nucleoplasm],R-HSA-109581,Apoptosis
1850045,CASP3(1-277) [cytosol],R-HSA-109581,Apoptosis
1850090,CASP3(176-277) [cytosol],R-HSA-2028269,Signaling by Hippo
1850091,CASP3(29-175) [cytosol],R-HSA-2028269,Signaling by Hippo
1850113,CASP3(176-277) [cytosol],R-HSA-5218859,Regulated Necrosis
1850114,CASP3(29-175) [cytosol],R-HSA-5218859,Regulated Necrosis
1850129,CASP3(176-277) [cytosol],R-HSA-73887,Death Receptor Signalling


In [106]:
dREACTpe.loc[dREACTpe[2].str.contains("p-\S*S22\S*-MAP2K2")]#"S221 NOT PRESENT

,2,3,5
1694908,"p-S222,S226-MAP2K2 [cytosol]",R-HSA-5683057,MAPK family signaling cascades
1694932,"p-S222,S226-MAP2K2 [cytosol]",R-HSA-9006934,Signaling by Receptor Tyrosine Kinases


In [94]:
dREACTpe.loc[dREACTpe[2].str.contains("^p-\S*S257\S*-MAP2K4")]

,2,3,5
1558821,"p-S257,T261-MAP2K4 [cytosol]",R-HSA-2262752,Cellular responses to stress


In [95]:
dREACTpe.loc[dREACTpe[2].str.contains("BTK")]

,2,3,5
1539418,BTK [cytosol],R-HSA-372790,Signaling by GPCR
1539442,BTK [cytosol],R-HSA-9716542,"Signaling by Rho GTPases, Miro GTPases and RHO..."


In [105]:
dREACTpe.loc[dREACTpe[2].str.contains("p-\S*Y41\S*-SRC")]#S418 NOT PRESENT

,2,3,5
1952627,SRC-1 [cytosol],R-HSA-354192,Integrin signaling
1952628,MyrG-p-Y419-SRC [plasma membrane],R-HSA-354192,Integrin signaling
1952629,"MyrG,p-Y530-SRC [plasma membrane]",R-HSA-354192,Integrin signaling
1952630,MyrG-SRC [plasma membrane],R-HSA-354192,Integrin signaling
1952633,MyrG-p-Y419-SRC [plasma membrane],R-HSA-372790,Signaling by GPCR
1952634,p-Y416-SRC [cytosol],R-HSA-372790,Signaling by GPCR
1952635,SRC [cytosol],R-HSA-372790,Signaling by GPCR
1952649,SRC-1 [cytosol],R-HSA-391160,Signal regulatory protein family interactions
1952711,MyrG-p-Y419-SRC [plasma membrane],R-HSA-5683057,MAPK family signaling cascades
1952712,"MyrG,p-Y530-SRC [plasma membrane]",R-HSA-5683057,MAPK family signaling cascades


In [107]:
dREACTpe.loc[dREACTpe[2].str.contains("p-\S*T37,T46\S*-EIF4EBP1")]

,2,3,5
1940322,"p-T37,T46-EIF4EBP1 [cytosol]",R-HSA-165159,MTOR signalling


In [109]:
dREACTpe.loc[dREACTpe[2].str.contains("p-\S*S8\S*-RB1")] #S811 AND S807 ARE SEPARATE TARGETS!

,2,3,5
1746356,p-S811-RB1 [nucleoplasm],R-HSA-69278,"Cell Cycle, Mitotic"
1746361,p-S807-RB1 [nucleoplasm],R-HSA-69278,"Cell Cycle, Mitotic"


In [113]:
dREACTpe.loc[dREACTpe[2].str.contains("p-\S*T308\S*-AKT1")] 

,2,3,5
1773553,"p-T308,S473-AKT1 [cytosol]",R-HSA-109581,Apoptosis
1773580,"p-T308,S473-AKT1 [cytosol]",R-HSA-157118,Signaling by NOTCH
1773600,"p-T308,S473-AKT1 [plasma membrane]",R-HSA-165159,MTOR signalling
1773612,"p-T308,S473-AKT1 [nucleoplasm]",R-HSA-195721,Signaling by WNT
1773639,"p-T308,S473-AKT1 [cytosol]",R-HSA-2262752,Cellular responses to stress
1773640,"p-T308,S473-AKT1 [plasma membrane]",R-HSA-354192,Integrin signaling
1773712,"p-T308,S473-AKT1 [cytosol]",R-HSA-69278,"Cell Cycle, Mitotic"
1773736,"p-T308,S473-AKT1 [nucleoplasm]",R-HSA-9006925,Intracellular signaling by second messengers
1773738,"p-T308,S473-AKT1 [cytosol]",R-HSA-9006925,Intracellular signaling by second messengers
1773743,p-T308-AKT1 [cytosol],R-HSA-9006925,Intracellular signaling by second messengers


In [114]:
dREACTpe.loc[dREACTpe[2].str.contains("p-\S*S473\S*-AKT1")] 

,2,3,5
1773553,"p-T308,S473-AKT1 [cytosol]",R-HSA-109581,Apoptosis
1773580,"p-T308,S473-AKT1 [cytosol]",R-HSA-157118,Signaling by NOTCH
1773600,"p-T308,S473-AKT1 [plasma membrane]",R-HSA-165159,MTOR signalling
1773612,"p-T308,S473-AKT1 [nucleoplasm]",R-HSA-195721,Signaling by WNT
1773639,"p-T308,S473-AKT1 [cytosol]",R-HSA-2262752,Cellular responses to stress
1773640,"p-T308,S473-AKT1 [plasma membrane]",R-HSA-354192,Integrin signaling
1773712,"p-T308,S473-AKT1 [cytosol]",R-HSA-69278,"Cell Cycle, Mitotic"
1773736,"p-T308,S473-AKT1 [nucleoplasm]",R-HSA-9006925,Intracellular signaling by second messengers
1773738,"p-T308,S473-AKT1 [cytosol]",R-HSA-9006925,Intracellular signaling by second messengers
1773740,p-S473-AKT1 [plasma membrane],R-HSA-9006925,Intracellular signaling by second messengers


In [115]:
dREACTpe.loc[dREACTpe[2].str.contains("p-\S*S133\S*-CREB1")]

,2,3,5
1671441,p-S133-CREB1 [nucleoplasm],R-HSA-157118,Signaling by NOTCH
1671504,p-S133-CREB1 [nucleoplasm],R-HSA-2262752,Cellular responses to stress
1671507,p-S133-CREB1 [nucleoplasm],R-HSA-372790,Signaling by GPCR
1671569,p-S133-CREB1 [nucleoplasm],R-HSA-9006925,Intracellular signaling by second messengers
1671572,p-S133-CREB1 [nucleoplasm],R-HSA-9006931,Signaling by Nuclear Receptors
1671575,p-S133-CREB1 [nucleoplasm],R-HSA-9006934,Signaling by Receptor Tyrosine Kinases


In [117]:
dREACTpe.loc[dREACTpe[2].str.contains("^SMAD(1|5|9)")]

/tmp/ipykernel_18924/1119892278.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  dREACTpe.loc[dREACTpe[2].str.contains("^SMAD(1|5|9)")]


,2,3,5
1654583,SMAD5 [cytosol],R-HSA-9006936,Signaling by TGFB family members
1675875,SMAD9 [cytosol],R-HSA-9006936,Signaling by TGFB family members
1884467,SMAD1 [cytosol],R-HSA-9006936,Signaling by TGFB family members


In [119]:
dREACTpe.loc[dREACTpe[2].str.contains("RELA")]

,2,3,5
1896422,RELA [nucleoplasm],R-HSA-2262752,Cellular responses to stress
1896474,RELA [nucleoplasm],R-HSA-73887,Death Receptor Signalling
1896475,RELA [cytosol],R-HSA-73887,Death Receptor Signalling
